In [2]:
# Imports 
import os
import dask.dataframe as dd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from lightgbm import LGBMRegressor, log_evaluation
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error  
from sklearn.model_selection import TimeSeriesSplit  

print('Ok - import')

Ok - import


In [11]:
# ---------------- model treningu LGBMRegressor ----------------
# Parametry modelu 
params = {
    'n_estimators': 200,           # Liczba drzew
    'learning_rate': 0.05,         # Współczynnik uczenia
    'num_leaves': 50,              # Liczba liści
    'feature_fraction': 0.9,       # Użycie 90% cech w każdym drzewie
    'subsample': 0.9,              # Losowanie 90% próbek do każdego drzewa
    'max_depth': 7,                # Maksymalna głębokość drzewa
    'min_child_samples': 20,       # Minimalna liczba próbek w liściu
}

# Inicjalizacja modelu
model = LGBMRegressor(**params)
# ---------------- model treningu LGBMRegressor ----------------

# Zmienne globalne nootbooka
competition_path = "M:/PycharmProjects/ds-zt-LGBMRegressor-main/kaggle-dtset"  # Ścieżka do danych konkursowych
batch_size = 1000000 # Rozmiar batcha treningowych
percent_of_dataset = 0.25 # Ile procent dataset używamy

print('Ok - param')

Ok - param


In [12]:
# Wczytanie danych przy użyciu Dask i próbkowanie
data_path = os.path.join(competition_path, "train.parquet")

features = [f'feature_{i:02}' for i in range(79)]
target = 'responder_6'
weights = 'weight'

ddf = dd.read_parquet(data_path, columns=features + [target, weights])
data = ddf.sample(frac=percent_of_dataset, random_state=42).compute()

# Usuwanie braków
data = data.dropna(subset=features + [target, weights])
print(f"Próbka danych w wymiarach: {data.shape}")

# Przygotowanie cech, etykiet i wag
X = data[features]
y = data[target]
sample_weights = data[weights]
num_batches = len(X) // batch_size + 1  # Liczba batchów

# funcja do przygotowania batch
def batch_generator(X, y, sample_weights, batch_size):
    for i in range(0, len(X), batch_size):
        yield (
            X.iloc[i:i + batch_size],
            y.iloc[i:i + batch_size],
            sample_weights.iloc[i:i + batch_size]
        )

print(f"num_batches: {num_batches}")
print('Ok - data')

Próbka danych w wymiarach: (8841701, 81)
num_batches: 9
Ok - data


In [14]:
# Trening modelu batchami
for i, (X_batch, y_batch, weights_batch) in enumerate(batch_generator(X, y, sample_weights, batch_size)):
    if i == 0:
        model.fit(X_batch, y_batch, sample_weight=weights_batch) # Trening od zera na pierwszym batchu
    else:
        model.fit(X_batch, y_batch, sample_weight=weights_batch, init_model=model.booster_) # Kontynuacja nauki na kolejnych batchach

    # Predykcja i monitorowanie błędu
    y_pred = model.predict(X_batch)
    mse = mean_squared_error(y_batch, y_pred)
    print(f"Batch {i + 1} - MSE: {mse:.4f}")

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.247045 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19385
[LightGBM] [Info] Number of data points in the train set: 1000000, number of used features: 79
[LightGBM] [Info] Start training from score -0.006271
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
Batch 1 - MSE: 0.7700
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.247045 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19385
[LightGBM] [Info] Number of data points in the train set: 1000000, number of used features: 79
[LightGBM] [Info] Start training from score -0.006271
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
Batch 1 - MSE: 0.7700
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.258650 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19346
[LightGBM] [Info] Number of data points in the train set: 1000000, number of used features: 79
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
Batch 2 - MSE: 0.7501
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.257080 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19344
[LightGBM] [Info] Number of data points in the train set: 1000000, number of used features: 79
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
Batch 3 - MSE: 0.6806
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.258321 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19339
[LightGBM] [Info] Number of data points in the train set: 1000000, number of used features: 79
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
Batch 4 - MSE: 0.5156
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.265642 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19348
[LightGBM] [Info] Number of data points in the train set: 1000000, number of used features: 79
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
Batch 5 - MSE: 0.7888
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.248696 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19326
[LightGBM] [Info] Number of data points in the train set: 1000000, number of used features: 79
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
Batch 6 - MSE: 0.5006
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.276170 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19330
[LightGBM] [Info] Number of data points in the train set: 1000000, number of used features: 79
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
Batch 7 - MSE: 0.6758
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.255388 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19325
[LightGBM] [Info] Number of data points in the train set: 1000000, number of used features: 79
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
Batch 8 - MSE: 0.4871
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.214358 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19310
[LightGBM] [Info] Number of data points in the train set: 841701, number of used features: 79
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
Batch 9 - MSE: 0.5239

In [10]:
# Walidacja krzyżowa z wykorzystaniem TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)  # 5-cio krotną walidację krzyżową
cv_results = []  # Przechowywanie wyników walidacji krzyżowej

for fold, (train_index, val_index) in enumerate(tscv.split(X), 1):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    
    # Trening modelu na danych treningowych
    model.fit(X_train, y_train, sample_weight=sample_weights.iloc[train_index], eval_metric='l2')
    
    # Predykcja na danych walidacyjnych
    y_val_pred = model.predict(X_val)
    
    # Obliczamy błąd na zbiorze walidacyjnym
    mse_val = mean_squared_error(y_val, y_val_pred)
    cv_results.append(mse_val)
    
    # Wypisujemy tylko numer walidacji i MSE
    print(f"Walidacja {fold} - MSE: {mse_val:.4f}")

# Podsumowanie wyników walidacji krzyżowej
mean_mse = np.mean(cv_results)
print(f"Średni MSE po walidacji krzyżowej: {mean_mse:.4f}")

print('Ok - test')

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.357829 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19426
[LightGBM] [Info] Number of data points in the train set: 1473621, number of used features: 79
[LightGBM] [Info] Start training from score -0.006848
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
Walidacja 1 - MSE: 0.7292
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGB

In [36]:
# Definicja funkcji wagowanego R²
def weighted_r2(y_true, y_pred, sample_weight):
    numerator = np.sum(sample_weight * (y_true - y_pred) ** 2)
    denominator = np.sum(sample_weight * y_true ** 2)
    return 1 - numerator / denominator if denominator != 0 else 0

weighted_r2_scorer = make_scorer(weighted_r2, greater_is_better=True, needs_proba=False)

In [15]:
# Parametry do optymalizacji
param_dist = {
    'n_estimators': [100, 200, 500],
    'learning_rate': [0.001, 0.005, 0.01],  # Zmniejszenie learning_rate
    'num_leaves': [31, 50, 100],
    'feature_fraction': [0.8, 0.9, 1.0],
    'subsample': [0.8, 0.9, 1.0],
    'max_depth': [-1, 7, 15],
}

# Model
lgbm_model = LGBMRegressor(random_state=42, verbose=1)  # Ustawienie verbose=1, aby wyświetlać postęp

# Callback do monitorowania logów podczas trenowania
callbacks = [log_evaluation(period=10)]  # Logowanie postępu co 10 iteracji

# RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=lgbm_model,
    param_distributions=param_dist,
    n_iter=20,
    scoring=weighted_r2_scorer,
    cv=tscv,
    n_jobs=-1,
    verbose=2  # Ustawienie verbose=2, ZEBY ZOBGACZYC SZCZEGOLY
)

# Dopasowanie modelu
random_search.fit(X_train_scaled, y_train, sample_weight=w_train, callbacks=callbacks)

# Najlepsze parametry
print("Najlepsze parametry:", random_search.best_params_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


KeyboardInterrupt: 